### Tópicos Especiais em Inteligência Artificial
### Professor Ciniro Nametala - IFMG

## Sistema de Perguntas e Respostas com Large Language Models (LLMs)

Neste trabalho vamos utilizar um modelo de linguagem pré-treinado para criar um sistema de perguntas e respostas (*Question Answering - Q&A*). O modelo utilizado é o **BERT** (*Bidirectional Encoder Representations from Transformers*), uma arquitetura de rede neural baseada em *Transformers* que revolucionou o processamento de linguagem natural (NLP).

O modelo específico utilizado é o `pierreguillou/bert-base-cased-squad-v1.1-portuguese`, que foi pré-treinado em português e fine-tunado no dataset SQuAD (Stanford Question Answering Dataset) adaptado para português. Este modelo é capaz de:

1. Receber um **contexto** (texto de referência)
2. Receber uma **pergunta** sobre o contexto
3. Extrair a **resposta** diretamente do texto

A arquitetura BERT utiliza o mecanismo de *self-attention* para capturar relações bidirecionais entre palavras, permitindo uma compreensão profunda do contexto textual.

**Observação:** Este notebook requer que o modelo seja pré-baixado. Execute o script `LLM_download_modelo_bert_qa.py` antes de usar este notebook e garanta que o modelo será baixado para pasta `modelo_bert_qa_pt/` (a pasta será criada no momento da execução do script).

## 1. Preparação do ambiente

### 1.1 Configurações de ambiente

In [1]:
#funcao para deixar o jupyter com celulas preenchendo toda a tela
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#instalacao de pacotes necessarios (executar apenas uma vez)
#!pip install transformers torch accelerate sentencepiece

### 1.2 Importação de pacotes

In [3]:
#importacao de bibliotecas

#bibliotecas basicas
import os
import warnings
warnings.filterwarnings('ignore')

#bibliotecas para trabalhar com dados
import numpy as np
import pandas as pd

#biblioteca para tocar sons
import pygame

#bibliotecas para llm
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

pygame 2.6.1 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 1.3 Verificando versões

In [4]:
#verificacao de versoes
import transformers
print('pytorch:', torch.__version__)
print('transformers:', transformers.__version__)

pytorch: 2.9.1
transformers: 4.57.5


### 1.4 Checagem de GPU/MPS

In [5]:
#checagem de dispositivo disponivel
def checarDispositivo():
    #verifica se cuda esta disponivel (nvidia)
    if torch.cuda.is_available():
        dispositivo = 'cuda'
        nome = torch.cuda.get_device_name(0)
    #verifica se mps esta disponivel (apple silicon)
    elif torch.backends.mps.is_available():
        dispositivo = 'mps'
        nome = 'Apple Silicon (MPS)'
    else:
        dispositivo = 'cpu'
        nome = 'CPU'
    
    print(f'dispositivo: {dispositivo}')
    print(f'nome: {nome}')
    return dispositivo

dispositivo = checarDispositivo()

dispositivo: mps
nome: Apple Silicon (MPS)


## 2. Carregamento do modelo pré-treinado

In [6]:
#diretorio do modelo pre-baixado
diretorio_modelo = './modelo_bert_qa_pt'

#verificar se o modelo foi baixado
if not os.path.exists(diretorio_modelo):
    print('ERRO: modelo nao encontrado!')
    print(f'execute primeiro o script: download_modelo_bert_qa.py')
    print(f'diretorio esperado: {os.path.abspath(diretorio_modelo)}')
else:
    print(f'modelo encontrado em: {diretorio_modelo}')

modelo encontrado em: ./modelo_bert_qa_pt


In [7]:
#nome do modelo original (para referencia)
nome_modelo = 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'

#carregar tokenizer do diretorio local
print('carregando tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(diretorio_modelo)
print('tokenizer carregado')

#carregar modelo do diretorio local
print('carregando modelo...')
modelo = AutoModelForQuestionAnswering.from_pretrained(diretorio_modelo)
print('modelo carregado')

carregando tokenizer...
tokenizer carregado
carregando modelo...
modelo carregado


In [8]:
#mover modelo para o dispositivo apropriado
modelo = modelo.to(dispositivo)
print(f'modelo movido para: {dispositivo}')

modelo movido para: mps


In [9]:
#criar pipeline de question answering
qa_pipeline = pipeline(
    'question-answering',
    model=modelo,
    tokenizer=tokenizer,
    device=dispositivo
)
print('pipeline criado com sucesso')

Device set to use mps


pipeline criado com sucesso


## 3. Configurações do experimento

In [10]:
#configuracoes de exibicao
exibir_tokens = True
exibir_detalhes = True
tocar_som = True

#inicializar pygame para sons
if tocar_som:
    pygame.mixer.init()

In [11]:
#funcao para tocar som de notificacao
def tocarSom(frequencia=440, duracao=200):
    if tocar_som:
        try:
            sample_rate = 44100
            n_samples = int(sample_rate * duracao / 1000)
            t = np.linspace(0, duracao / 1000, n_samples, False)
            wave = np.sin(2 * np.pi * frequencia * t) * 0.3
            wave = (wave * 32767).astype(np.int16)
            stereo_wave = np.column_stack((wave, wave))
            sound = pygame.sndarray.make_sound(stereo_wave)
            sound.play()
            pygame.time.wait(duracao)
        except:
            pass

## 4. Análise do modelo

### 4.1 Arquitetura do modelo

In [12]:
#exibir arquitetura do modelo
print('='*60)
print('ARQUITETURA DO MODELO BERT')
print('='*60)
print(f'nome: {nome_modelo}')
print(f'tipo: {modelo.config.model_type}')
print(f'camadas (layers): {modelo.config.num_hidden_layers}')
print(f'dimensao oculta (hidden size): {modelo.config.hidden_size}')
print(f'cabecas de atencao (attention heads): {modelo.config.num_attention_heads}')
print(f'tamanho do vocabulario: {modelo.config.vocab_size}')
print(f'tamanho maximo de sequencia: {modelo.config.max_position_embeddings}')
print('='*60)

ARQUITETURA DO MODELO BERT
nome: pierreguillou/bert-base-cased-squad-v1.1-portuguese
tipo: bert
camadas (layers): 12
dimensao oculta (hidden size): 768
cabecas de atencao (attention heads): 12
tamanho do vocabulario: 29794
tamanho maximo de sequencia: 512


In [13]:
#contar parametros do modelo
def contarParametros(model):
    total = sum(p.numel() for p in model.parameters())
    treinaveis = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, treinaveis

total_params, treinaveis_params = contarParametros(modelo)
print(f'total de parametros: {total_params:,}')
print(f'parametros treinaveis: {treinaveis_params:,}')
print(f'tamanho aproximado: {total_params * 4 / 1024 / 1024:.1f} MB')

total de parametros: 108,334,082
parametros treinaveis: 108,334,082
tamanho aproximado: 413.3 MB


### 4.2 Tokenizer e vocabulário

In [14]:
#informacoes do tokenizer
print('='*60)
print('INFORMACOES DO TOKENIZER')
print('='*60)
print(f'tipo: {tokenizer.__class__.__name__}')
print(f'tamanho do vocabulario: {tokenizer.vocab_size}')
print(f'tamanho maximo: {tokenizer.model_max_length}')
print(f'tokens especiais: {tokenizer.all_special_tokens}')
print('='*60)

INFORMACOES DO TOKENIZER
tipo: BertTokenizerFast
tamanho do vocabulario: 29794
tamanho maximo: 1000000000000000019884624838656
tokens especiais: ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [15]:
#exibir alguns tokens do vocabulario
if exibir_tokens:
    vocab = tokenizer.get_vocab()
    vocab_ordenado = sorted(vocab.items(), key=lambda x: x[1])
    
    print('20 tokens aleatórios do vocabulario:')
    for token, idx in vocab_ordenado[10190:10210]:
        print(f'  {idx}: {repr(token)}')

20 tokens aleatórios do vocabulario:
  10190: '##iterrâneo'
  10191: 'Comércio'
  10192: 'Mi'
  10193: '##idado'
  10194: 'reag'
  10195: 'sons'
  10196: 'Integra'
  10197: 'hectares'
  10198: 'substituiu'
  10199: 'descendente'
  10200: 'rem'
  10201: 'lembra'
  10202: 'ortodo'
  10203: 'líquido'
  10204: 'Christian'
  10205: 'laboratório'
  10206: 'Your'
  10207: '##íferos'
  10208: 'célula'
  10209: 'cérebro'


### 4.3 Exemplo de tokenização

In [16]:
#demonstracao de tokenizacao
texto_exemplo = 'O Instituto Federal de Minas Gerais oferece cursos de tecnologia.'

print('texto original:')
print(f'  "{texto_exemplo}"')
print()

#tokenizar
tokens = tokenizer.tokenize(texto_exemplo)
print(f'tokens ({len(tokens)}):')
print(f'  {tokens}')
print()

#converter para ids
token_ids = tokenizer.encode(texto_exemplo)
print(f'token ids ({len(token_ids)}):')
print(f'  {token_ids}')
print()

#decodificar de volta
texto_decodificado = tokenizer.decode(token_ids)
print('texto decodificado:')
print(f'  "{texto_decodificado}"')

texto original:
  "O Instituto Federal de Minas Gerais oferece cursos de tecnologia."

tokens (11):
  ['O', 'Instituto', 'Federal', 'de', 'Minas', 'Gerais', 'oferece', 'cursos', 'de', 'tecnologia', '.']

token ids (13):
  [101, 231, 2900, 2528, 125, 3474, 4192, 6158, 4736, 125, 4277, 119, 102]

texto decodificado:
  "[CLS] O Instituto Federal de Minas Gerais oferece cursos de tecnologia. [SEP]"


## 5. Demonstração de Q&A

### 5.1 Funções auxiliares

In [17]:
#funcao principal de perguntas e respostas
def responderPergunta(contexto, pergunta, exibir=True):
    #executar inferencia
    resultado = qa_pipeline(question=pergunta, context=contexto)
    
    #extrair informacoes
    resposta = resultado['answer']
    score = resultado['score']
    inicio = resultado['start']
    fim = resultado['end']
    
    if exibir:
        print('='*60)
        print('PERGUNTA:')
        print(f'  {pergunta}')
        print()
        print('RESPOSTA:')
        print(f'  {resposta}')
        print()
        if exibir_detalhes:
            print('DETALHES:')
            print(f'  confianca: {score:.4f} ({score*100:.1f}%)')
            print(f'  posicao no texto: caracteres {inicio} a {fim}')
        print('='*60)
    
    tocarSom(880, 100)
    
    return resultado

In [18]:
#funcao para multiplas perguntas
def responderMultiplasPerguntas(contexto, perguntas):
    print('='*60)
    print('CONTEXTO:')
    print(f'  {contexto[:200]}...' if len(contexto) > 200 else f'  {contexto}')
    print('='*60)
    print()
    
    resultados = []
    for pergunta in perguntas:
        resultado = responderPergunta(contexto, pergunta, exibir=True)
        resultados.append(resultado)
        print()
    
    return resultados

### 5.2 Testes com textos de exemplo

In [19]:
#texto de exemplo sobre o ifmg
contexto_ifmg = """
O Instituto Federal de Educação, Ciência e Tecnologia de Minas Gerais (IFMG) é uma 
instituição de ensino público federal brasileira. Foi criado em 2008 através da Lei 
11.892. O IFMG possui 18 campi distribuídos pelo estado de Minas Gerais, oferecendo 
cursos técnicos, graduação e pós-graduação. A reitoria está localizada em Belo Horizonte. 
O instituto oferece cursos nas áreas de tecnologia, engenharia, administração, 
licenciaturas e outras. O campus Bambuí é um dos mais antigos, tendo origem na antiga 
Escola Agrícola de Bambuí fundada em 1956.
"""

perguntas_ifmg = [
    'Quando o IFMG foi criado?',
    'Quantos campi o IFMG possui?',
    'Onde está localizada a reitoria?',
    'Quando foi fundada a Escola Agrícola de Bambuí?'
]

resultados = responderMultiplasPerguntas(contexto_ifmg, perguntas_ifmg)

CONTEXTO:
  
O Instituto Federal de Educação, Ciência e Tecnologia de Minas Gerais (IFMG) é uma 
instituição de ensino público federal brasileira. Foi criado em 2008 através da Lei 
11.892. O IFMG possui 18 campi...

PERGUNTA:
  Quando o IFMG foi criado?

RESPOSTA:
  2008

DETALHES:
  confianca: 0.9825 (98.2%)
  posicao no texto: caracteres 149 a 153

PERGUNTA:
  Quantos campi o IFMG possui?

RESPOSTA:
  18

DETALHES:
  confianca: 0.9400 (94.0%)
  posicao no texto: caracteres 192 a 194

PERGUNTA:
  Onde está localizada a reitoria?

RESPOSTA:
  Belo Horizonte

DETALHES:
  confianca: 0.9670 (96.7%)
  posicao no texto: caracteres 329 a 343

PERGUNTA:
  Quando foi fundada a Escola Agrícola de Bambuí?

RESPOSTA:
  1956

DETALHES:
  confianca: 0.9965 (99.6%)
  posicao no texto: caracteres 550 a 554



## 6. Experimentos interativos

### 6.1 Notícias

In [20]:
#texto de noticia
contexto_noticia = """
A seleção brasileira de futebol conquistou o pentacampeonato mundial em 2002, na Copa 
do Mundo realizada no Japão e na Coreia do Sul. O técnico Luiz Felipe Scolari comandou 
a equipe que tinha Ronaldo como principal artilheiro, com 8 gols marcados no torneio. 
A final foi disputada contra a Alemanha no dia 30 de junho, em Yokohama, com vitória 
brasileira por 2 a 0. Os gols foram marcados por Ronaldo, que se recuperava de uma 
grave lesão no joelho. Outros jogadores importantes daquela campanha foram Rivaldo, 
Ronaldinho Gaúcho, Cafu e Roberto Carlos.
"""

perguntas_noticia = [
    'Quem foi o técnico da seleção em 2002?',
    'Quantos gols Ronaldo marcou?',
    'Qual foi o placar da final?',
    'Onde foi disputada a final?',
    'Contra quem o Brasil jogou a final?'
]

resultados = responderMultiplasPerguntas(contexto_noticia, perguntas_noticia)

CONTEXTO:
  
A seleção brasileira de futebol conquistou o pentacampeonato mundial em 2002, na Copa 
do Mundo realizada no Japão e na Coreia do Sul. O técnico Luiz Felipe Scolari comandou 
a equipe que tinha Ronal...

PERGUNTA:
  Quem foi o técnico da seleção em 2002?

RESPOSTA:
  Luiz Felipe Scolari

DETALHES:
  confianca: 0.9920 (99.2%)
  posicao no texto: caracteres 146 a 165

PERGUNTA:
  Quantos gols Ronaldo marcou?

RESPOSTA:
  8

DETALHES:
  confianca: 0.8709 (87.1%)
  posicao no texto: caracteres 234 a 235

PERGUNTA:
  Qual foi o placar da final?

RESPOSTA:
  2 a 0

DETALHES:
  confianca: 0.8865 (88.6%)
  posicao no texto: caracteres 364 a 369

PERGUNTA:
  Onde foi disputada a final?

RESPOSTA:
  Yokohama

DETALHES:
  confianca: 0.2073 (20.7%)
  posicao no texto: caracteres 326 a 334

PERGUNTA:
  Contra quem o Brasil jogou a final?

RESPOSTA:
  Alemanha

DETALHES:
  confianca: 0.8444 (84.4%)
  posicao no texto: caracteres 294 a 302



### 6.2 Textos científicos

In [21]:
#texto cientifico sobre inteligencia artificial
contexto_ia = """
A Inteligência Artificial (IA) é um campo da ciência da computação que busca criar 
sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. 
O termo foi cunhado por John McCarthy em 1956, durante a conferência de Dartmouth. 
As principais subáreas da IA incluem aprendizado de máquina, processamento de linguagem 
natural, visão computacional e robótica. O aprendizado profundo (deep learning) é uma 
técnica de aprendizado de máquina que utiliza redes neurais artificiais com múltiplas 
camadas. O modelo GPT-3, lançado pela OpenAI em 2020, possui 175 bilhões de parâmetros 
e representa um marco no desenvolvimento de modelos de linguagem.
"""

perguntas_ia = [
    'Quem cunhou o termo Inteligência Artificial?',
    'Em que ano foi a conferência de Dartmouth?',
    'O que é deep learning?',
    'Quantos parâmetros tem o GPT-3?',
    'Quem lançou o GPT-3?'
]

resultados = responderMultiplasPerguntas(contexto_ia, perguntas_ia)

CONTEXTO:
  
A Inteligência Artificial (IA) é um campo da ciência da computação que busca criar 
sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. 
O termo foi cunhado por John Mc...

PERGUNTA:
  Quem cunhou o termo Inteligência Artificial?

RESPOSTA:
  John McCarthy

DETALHES:
  confianca: 0.9990 (99.9%)
  posicao no texto: caracteres 193 a 206

PERGUNTA:
  Em que ano foi a conferência de Dartmouth?

RESPOSTA:
  1956

DETALHES:
  confianca: 0.9964 (99.6%)
  posicao no texto: caracteres 210 a 214

PERGUNTA:
  O que é deep learning?

RESPOSTA:
  aprendizado profundo

DETALHES:
  confianca: 0.8782 (87.8%)
  posicao no texto: caracteres 385 a 405

PERGUNTA:
  Quantos parâmetros tem o GPT-3?

RESPOSTA:
  175 bilhões

DETALHES:
  confianca: 0.8989 (89.9%)
  posicao no texto: caracteres 577 a 588

PERGUNTA:
  Quem lançou o GPT-3?

RESPOSTA:
  OpenAI

DETALHES:
  confianca: 0.9800 (98.0%)
  posicao no texto: caracteres 554 a 560



### 6.3 Texto livre (interativo)

In [22]:
#celula interativa - insira seu proprio contexto e pergunta

meu_contexto = """
Cole aqui o texto que deseja usar como contexto para as perguntas.
Pode ser uma notícia, um artigo, um trecho de livro ou qualquer texto.
"""

minha_pergunta = "Sua pergunta aqui?"

#descomente a linha abaixo para executar
#resultado = responderPergunta(meu_contexto, minha_pergunta)

## 7. Análise de confiança e limitações

### 7.1 Testando limites do modelo

In [23]:
#testando perguntas cujas respostas nao estao no texto
contexto_teste = """
O Brasil é o maior país da América do Sul, com uma população de aproximadamente 
215 milhões de habitantes. A capital do país é Brasília, fundada em 1960. 
O idioma oficial é o português.
"""

perguntas_teste = [
    'Qual é a capital do Brasil?',           #resposta no texto
    'Qual é a população do Brasil?',         #resposta no texto
    'Quem é o presidente do Brasil?',        #resposta nao esta no texto
    'Qual é a moeda do Brasil?'              #resposta nao esta no texto
]

print('TESTE DE LIMITES DO MODELO')
print('='*60)
print('observe como o modelo se comporta com perguntas cuja resposta')
print('nao esta presente no contexto fornecido')
print('='*60)
print()

for pergunta in perguntas_teste:
    resultado = responderPergunta(contexto_teste, pergunta)
    print()

TESTE DE LIMITES DO MODELO
observe como o modelo se comporta com perguntas cuja resposta
nao esta presente no contexto fornecido

PERGUNTA:
  Qual é a capital do Brasil?

RESPOSTA:
  Brasília

DETALHES:
  confianca: 0.9853 (98.5%)
  posicao no texto: caracteres 129 a 137

PERGUNTA:
  Qual é a população do Brasil?

RESPOSTA:
  215 milhões

DETALHES:
  confianca: 0.8053 (80.5%)
  posicao no texto: caracteres 82 a 93

PERGUNTA:
  Quem é o presidente do Brasil?

RESPOSTA:
  Brasília

DETALHES:
  confianca: 0.2278 (22.8%)
  posicao no texto: caracteres 129 a 137

PERGUNTA:
  Qual é a moeda do Brasil?

RESPOSTA:
  português

DETALHES:
  confianca: 0.8486 (84.9%)
  posicao no texto: caracteres 178 a 187



### 7.2 Comparativo de confiança

In [24]:
#gerar tabela comparativa de confianca
contexto_comp = """
Albert Einstein nasceu em 14 de março de 1879 em Ulm, na Alemanha. Ele desenvolveu 
a teoria da relatividade e ganhou o Prêmio Nobel de Física em 1921. Einstein emigrou 
para os Estados Unidos em 1933 e trabalhou no Instituto de Estudos Avançados em Princeton. 
Ele faleceu em 18 de abril de 1955.
"""

perguntas_comp = [
    ('Onde Einstein nasceu?', True),
    ('Quando Einstein ganhou o Nobel?', True),
    ('Para onde Einstein emigrou?', True),
    ('Qual era a cor favorita de Einstein?', False),
    ('Quantos filhos Einstein teve?', False)
]

print('COMPARATIVO DE CONFIANCA')
print('='*70)
print(f'{"pergunta":<40} {"resposta":<15} {"confianca":<10} {"no texto?"}')
print('='*70)

for pergunta, esperado in perguntas_comp:
    resultado = qa_pipeline(question=pergunta, context=contexto_comp)
    resposta = resultado['answer'][:12] + '...' if len(resultado['answer']) > 12 else resultado['answer']
    score = resultado['score']
    print(f'{pergunta:<40} {resposta:<15} {score:.4f}     {"sim" if esperado else "nao"}')

print('='*70)
tocarSom(660, 150)

COMPARATIVO DE CONFIANCA
pergunta                                 resposta        confianca  no texto?
Onde Einstein nasceu?                    Ulm, na Alem... 0.8927     sim
Quando Einstein ganhou o Nobel?          1921            0.9860     sim
Para onde Einstein emigrou?              Estados Unid... 0.9417     sim
Qual era a cor favorita de Einstein?     relatividade    0.0742     nao
Quantos filhos Einstein teve?            1933            0.1286     nao
